Importando as bibliotecas utilizadas ... 

In [1]:
import numpy as np
import pandas as pd
import pickle
import gensim
import spacy

D:\ANACONDA\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Inicializando o DataFrame da matriz dos conhecimentos, 
utilizando a função 'fillna' para consertar as células mescladas,
lendo a partir da linha 4
e lendo até a coluna 9 (excluso)

In [2]:
crea_df = pd.read_excel("C:\\Coding\\PIBITI-git-repo\\crea-tools\\Matriz_do_Conhecimento.xls", skiprows=4).iloc[:, :9].fillna(method="ffill")

Corrigindo valores que foram incorretamente preenchido pelo método fillna("ffill"):

In [3]:
crea_df = crea_df.replace({"TÓPICOS": np.NaN, "Nº DE ORDEM DOS TÓPICOS": np.NaN})

Salvando o Dataframe com o pickle

In [4]:
with open('crea_df_pickle.txt', 'wb') as f:
    pickle.dump(crea_df, f)

Abrindo o dataFrame em formato Binário e Imprimindo o DataFrame para fins de visualização:

In [5]:
with open('crea_df_pickle.txt', 'rb') as f:
    crea_df = pickle.load(f)

# seção de Construção de Edificações
subsection1_df = crea_df[ crea_df['SUB-SETOR'] == 'Construção de Edificações']

Importando as matérias que foram manualmente obtidas e salvas em um arquivo json

In [6]:
subjects_df = pd.read_json("C:\\Coding\\PIBITI-git-repo\\crea-tools\\MechatronicsEngeneeringSubjects.json")
subjects_df

,codigo,nome,ementa,conteudo
0,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,Princípios fundamentais de construção de progr...,
1,IFD0171,FISICA 1,Módulos 1: Unidades e grandezas físicas 2: Vet...,"I-MEDICAO: GRANDEZAS, PADROES E UNIDADES FISIC..."
2,IFD0173,FISICA 1 EXPERIMENTAL,MEDIDAS E ERROS. ANALISE GRAFICA. ATRITO. COLI...,I-CLASSIFICACAO DOS ERROS. CALCULO DE ERRO EXP...
3,MAT0025,CÁLCULO 1,"Funções de uma variável real, limite e continu...",1. Funções: conceito de função exemplo de funç...
4,ENM0133,INTRODUCAO A ENGENHARIA MECATRONICA,"A Universidade de Brasília. Engenharia, Contro...",
5,IQD0125,QUIMICA GERAL TEORICA,Abordagem conceitual dos princípios fundamenta...,1.Estrutura Atômica e a Lei Periódica: O Model...
6,IQD0126,QUIMICA GERAL EXPERIMENTAL,CARACTERIZACAO DA NATUREZA E DO PAPEL DAS INVE...,1) Noções Básicas sobre Segurança no Trabalho ...
7,ENM0131,DESENHO MECANICO ASSISTIDO POR COMPUTADOR 1,Objetivos da Disciplina: A disciplina objetiva...,- Normas de desenho Técnico- Introdução ao CAD...
8,EST0023,PROBABILIDADE E ESTATÍSTICA,"Análise Descritiva, Cálculo de Probabilidades,...",UNIDADE I - Análise Descritiva de Dados - Popu...
9,IFD0175,FISICA 2,Módulos 1: Equilíbrio e elasticidade 2: Gravit...,1-CONTEUDO TEORICO 1-DINAMICA DA ROTACAO 1.1- ...


Criando uma função de pre-processamento de texto que remove acentos, símbolos, números, stopwords;e deixa tudo em minúsculo. 

Recebe um texto no formato de string e retorna uma lista com de strings com as palavras do documento.

Todo: A lemmatização causou alguns conflitos, como "serie" -> "seriar" <- "seriam". Talvez seja melhor não usar os lemmas?(pelo menos nos stopwords). Adicionar Bigramas e Trigramas

In [7]:
from nltk.corpus import stopwords
nlp = spacy.load('pt_core_news_lg', disable=['parser', 'ner'])

stopWords_list = stopwords.words("portuguese")

# * adding roman numerals to stowords
stopWords_list += ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', 'referente', 'seguinte']

# * preprocessing stopWords with gensim.simple_preprocess
stopWords_list = gensim.utils.simple_preprocess(" ".join(stopWords_list), deacc=True, min_len=2, max_len=40)

#recombining stopWords list to a string object
# stopWords_str = " ".join(stopWords_list)

#spacy preprocessing
# stopWords_doc = nlp(stopWords_str)

#lemmatization
# lemmatized_stopWords_list = [token.lemma_ for token in stopWords_doc]

print(stopWords_list)

def preprocess(text):
    # * importing stopwords from nltk and spacy pipeline
    global nlp
    global lemmatized_stopWords_list

    # * preprocessing text with gensim.simple_preprocess
    text_list = gensim.utils.simple_preprocess(text, deacc=True, min_len=2, max_len=40)

    # * removing stopWords
    text_list = [word for word in text_list if not word in stopWords_list]

    # * recombining words of text to a string object
    text_str = " ".join(text_list)

    # * preprocessing with spacy
    text_doc = nlp(text_str)
    
    # * lemmatizing with lemma method from spacy and tokenization
    lemmatized_text_list = [token.lemma_ for token in text_doc]

    return lemmatized_text_list

['de', 'que', 'do', 'da', 'em', 'um', 'para', 'com', 'nao', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'ja', 'eu', 'tambem', 'so', 'pelo', 'pela', 'ate', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'voce', 'essa', 'num', 'nem', 'suas', 'meu', 'as', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nos', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'voces', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'esta', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estiveramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivessemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei

Pre-processando as matérias com Stopwords do NLTK e função do Gensim. E adicionando uma nova columa ao Dataframe que consiste em todo o texto da matéria

In [8]:
documents_list = []

for i, row in subjects_df.iterrows():

    # * reading values of each subject (row)
    subject_id = row["codigo"]
    name = row["nome"]
    syllabus = row["ementa"]
    content = row["conteudo"]

    # * combining them to create the subject document
    text = name + ' ' + syllabus + ' ' + content
    
    # * preprocessing
    preProcessedText = preprocess(text)
    documents_list.append(preProcessedText)

documents_series = pd.Series(documents_list, name="documento")

documents_df = pd.concat([subjects_df, documents_series], axis=1)

Salvando o Dataframe das matérias,  reabrindo-o e imprimindo o DataFrame para fins de visualização:

In [9]:
with open('documents_df.txt', 'wb') as f:
    pickle.dump(documents_df, f)

In [10]:
with open('documents_df.txt', 'rb') as f:
    documents_df = pickle.load(f)

documents_df

,codigo,nome,ementa,conteudo,documento
0,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,Princípios fundamentais de construção de progr...,,"[algoritmo, programacao, computador, principio..."
1,IFD0171,FISICA 1,Módulos 1: Unidades e grandezas físicas 2: Vet...,"I-MEDICAO: GRANDEZAS, PADROES E UNIDADES FISIC...","[fisica, modulos, unidade, grandeza, fisicas, ..."
2,IFD0173,FISICA 1 EXPERIMENTAL,MEDIDAS E ERROS. ANALISE GRAFICA. ATRITO. COLI...,I-CLASSIFICACAO DOS ERROS. CALCULO DE ERRO EXP...,"[fisica, experimental, medir, erro, analisar, ..."
3,MAT0025,CÁLCULO 1,"Funções de uma variável real, limite e continu...",1. Funções: conceito de função exemplo de funç...,"[calcular, funcoes, variavel, real, limitar, c..."
4,ENM0133,INTRODUCAO A ENGENHARIA MECATRONICA,"A Universidade de Brasília. Engenharia, Contro...",,"[introducao, engenhar, mecatronica, universida..."
5,IQD0125,QUIMICA GERAL TEORICA,Abordagem conceitual dos princípios fundamenta...,1.Estrutura Atômica e a Lei Periódica: O Model...,"[quimica, geral, teorica, abordagem, conceitua..."
6,IQD0126,QUIMICA GERAL EXPERIMENTAL,CARACTERIZACAO DA NATUREZA E DO PAPEL DAS INVE...,1) Noções Básicas sobre Segurança no Trabalho ...,"[quimica, geral, experimental, caracterizacao,..."
7,ENM0131,DESENHO MECANICO ASSISTIDO POR COMPUTADOR 1,Objetivos da Disciplina: A disciplina objetiva...,- Normas de desenho Técnico- Introdução ao CAD...,"[desenhar, mecanico, assistir, computador, obj..."
8,EST0023,PROBABILIDADE E ESTATÍSTICA,"Análise Descritiva, Cálculo de Probabilidades,...",UNIDADE I - Análise Descritiva de Dados - Popu...,"[probabilidade, estatistica, analisar, descrit..."
9,IFD0175,FISICA 2,Módulos 1: Equilíbrio e elasticidade 2: Gravit...,1-CONTEUDO TEORICO 1-DINAMICA DA ROTACAO 1.1- ...,"[fisica, modulos, equilibrio, elasticidade, gr..."


Salvando a coluna de documentos para tornar legível e facilitar a busca, assim podendo conferir se os lemmas estão satisfatórios e se condizem

In [11]:
with open("debbuging_docs.txt", "w") as f:
    f.write( documents_df['documento'].to_json())

construindo o Corpus, vetorizando os documentos com o bag-of-words do gensim.

Adicionando os documentos extras do wikipedia

In [56]:
from gensim import corpora
import json
lemmatizedData = documents_df["documento"].tolist()

with open('wikipediaData.json', 'r+') as f:
    extra_list = json.load(f)

extraData = [preprocess(text) for text in extra_list]
print(extraData)

lemmatizedData += extraData
# * gensim dictionary object, which will track each word to its respective id
id2wordDict = corpora.Dictionary(lemmatizedData)

# * gensim doc2bow method to map each word to a integer id and its respective frequency
corpus = [id2wordDict.doc2bow(text) for text in lemmatizedData]

# * list of list of tuples (id of a word, frequency)
corpus

[['eletromagnetismo', 'electromagnetismo', 'ramo', 'fisica', 'estudar', 'unificadamente', 'fenomenos', 'eletricidade', 'magnetismo', 'teoria', 'basear', 'conceito', 'campar', 'eletromagnetico', 'interacao', 'conjunto', 'campo', 'eletrico', 'magnetico', 'tal', 'interacao', 'regido', 'quatro', 'equacoes', 'maxwell', 'campar', 'magnetico', 'resultar', 'movimentar', 'carga', 'eletricas', 'resultar', 'corrente', 'eletrica', 'campar', 'magnetico', 'poder', 'resultar', 'forcar', 'eletromagnetica', 'associar', 'imo', 'variacao', 'fluxo', 'magnetico', 'resultar', 'campar', 'eletrico', 'fenomeno', 'conhecer', 'inducao', 'eletromagnetica', 'mecanismo', 'utilizar', 'gerador', 'eletricos', 'motor', 'transformador', 'tensao', 'maneiro', 'semelhante', 'variacao', 'campar', 'eletrico', 'gerar', 'campar', 'magnetico', 'dever', 'interdependencia', 'campar', 'eletrico', 'campar', 'magnetico', 'fazer', 'sentir', 'falar', 'unica', 'entidade', 'chamar', 'campar', 'eletromagnetico', 'teoria', 'eletromagnetic

[[(0, 1),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 3),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 3),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 3),
  (39, 3),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 2),
  (51, 1),
  (52, 1)],
 [(35, 1),
  (37, 1),
  (45, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 3),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 2),
  (61, 1),
  (62, 5),
  (63, 2),
  (64, 3),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 3),
  (69, 1),
  (70, 5),
  (71, 1),
  (72, 6),
  (73, 1),
  (74, 1),
  (75, 2),
  (76, 4),
  (77, 1),
  (78, 1),
  (79, 3),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 10),
  (84, 1),
  (85, 1),
  (86, 3),
  (87, 

Utilizando tf-idf do gensim para descobrir as palavras mais importantes e dar mais pesos a elas;

tupla(int, int) -> tupla(int, float)

td-idf(term_i, document_j) = freq(i,j) * log2 ( inverse_document_frequency(i) )

freq(i, j) = total occurances of i in j / total words in j

inverse_document_frequency(i) = total documents / documents that have at least one occurance of i 

O parâmetro id2word recebe o dicionário que mapeia as palavras com os respectivos IDs.

In [57]:
tfidf_model = gensim.models.TfidfModel(corpus, id2word=id2wordDict)

LSI, pega a matriz do corpus e a decompõe utilizando o SVD. Das três matrizes criadas, utiliza-se apenas o right singular vectors que representa a relação entre os tópicos latentes com as palavras.

O parâmetro id2word recebe o dicionário que mapeia as palavras com os respectivos IDs. E o parâmetro power_iters define o número de iterações para treinamento do modelo, e, portanto, quanto maior o valor, mais acurado e mais devagar vai ser o treino modelo

In [58]:
lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=id2wordDict, num_topics=100, power_iters=10)

lsi_model.print_topics()

[(0,
  '-0.247*"circuito" + -0.182*"analisar" + -0.177*"transformar" + -0.138*"seriar" + -0.136*"fourier" + -0.131*"laplace" + -0.128*"movimentar" + -0.127*"energia" + -0.117*"tecnicas" + -0.114*"rigidos"'),
 (1,
  '-0.285*"circuito" + 0.206*"movimentar" + 0.195*"rigidos" + 0.189*"corpo" + 0.171*"momento" + 0.168*"energia" + -0.165*"transformar" + -0.159*"analisar" + -0.143*"senoidal" + -0.136*"tecnicas"'),
 (2,
  '0.194*"transformar" + 0.184*"fourier" + 0.183*"seriar" + -0.180*"circuito" + 0.174*"funcoes" + -0.155*"combinacionais" + -0.155*"conversor" + -0.155*"sequencial" + 0.151*"laplace" + 0.143*"funcao"'),
 (3,
  '0.176*"combinacionais" + 0.176*"sequencial" + 0.176*"conversor" + -0.167*"analisar" + -0.159*"senoidal" + -0.145*"regime" + -0.139*"permanente" + 0.138*"funcoes" + 0.135*"projeto" + 0.133*"derivar"'),
 (4,
  '0.234*"magnetico" + 0.222*"c" + 0.211*"campar" + 0.193*"motor" + -0.176*"transformar" + 0.171*"corrente" + 0.153*"eletrico" + 0.145*"gerador" + -0.142*"fourier" + -

left singular vector -> term-to-topic matrix (não será utilizado)

In [59]:
# * U Matrix
print(np.shape(lsi_model.projection.u))
lsi_model.projection.u

(1636, 43)


array([[-0.00638063,  0.00048412, -0.00121582, ...,  0.00028074,
        -0.00164631,  0.0025738 ],
       [-0.00638063,  0.00048412, -0.00121582, ...,  0.00028074,
        -0.00164631,  0.0025738 ],
       [-0.02442885, -0.02679765, -0.04660947, ..., -0.00173222,
        -0.01200748, -0.00077264],
       ...,
       [-0.00546482,  0.00297944,  0.01250957, ..., -0.0024042 ,
        -0.00088566,  0.00089635],
       [-0.00546482,  0.00297944,  0.01250957, ..., -0.0024042 ,
        -0.00088566,  0.00089635],
       [-0.00546482,  0.00297944,  0.01250957, ..., -0.0024042 ,
        -0.00088566,  0.00089635]])

singular values -> "impacto" (abrangência?) de cada tópico

Pode-se escolher um valor de corte para reduzir o valor do parametro num_topics, aumentando, assim, o desempenho de tempo. O número de tópicos latentes não tem como ser maior que o numero de documentos

In [60]:
print("número de documentos/ementas/matérias:", len(subjects_df))
print("topicos latentes encontrados com um \'impacto\' alto:", len(lsi_model.projection.s))

# * S matrix (sigma)
print(np.shape(lsi_model.projection.s))
lsi_model.projection.s

número de documentos/ementas/matérias: 41
topicos latentes encontrados com um 'impacto' alto: 43
(43,)


array([1.67236578, 1.47060563, 1.36199556, 1.30352428, 1.27471159,
       1.21438474, 1.17535384, 1.15370732, 1.12908749, 1.11286999,
       1.09959858, 1.08575388, 1.0526698 , 1.05003736, 1.03980864,
       1.01262204, 1.00752122, 1.00134126, 0.99587728, 0.98113628,
       0.97281854, 0.96949327, 0.9548277 , 0.94779919, 0.93785097,
       0.93664676, 0.90743801, 0.88761816, 0.88172451, 0.86826849,
       0.8657352 , 0.84598568, 0.82867729, 0.81314204, 0.80368807,
       0.79429279, 0.76936621, 0.75003901, 0.72799195, 0.69333949,
       0.53428918, 0.4299394 , 0.40272719])

right singular vectors -> document-to-topic matrix (Não é diretamente calculado e armazenado pois pode ser muito grande devido a quantidade de documentos -> num_topic x documents)

Será o documento armazenado e utilizado para fazer as queries de similaridade

In [17]:
V_matrix = gensim.matutils.corpus2dense(lsi_model[tfidf_model[corpus]], len(lsi_model.projection.s)).T / lsi_model.projection.s
print(np.shape(V_matrix))
# * V or V^T matrix
# * representado por lsi_model[x]
V_matrix

(43, 43)


array([[-0.07372696,  0.00491901, -0.01144136, ...,  0.00103637,
        -0.00489046,  0.00716172],
       [-0.23879961,  0.33900054, -0.14080346, ..., -0.02107091,
         0.01990575, -0.03068936],
       [-0.16897339,  0.23988228, -0.08306264, ...,  0.00126755,
        -0.00489613,  0.00309715],
       ...,
       [-0.02689805,  0.01678192, -0.03255246, ..., -0.00067114,
         0.00092244,  0.00080958],
       [-0.1267368 ,  0.07463915, -0.02309095, ...,  0.01679241,
         0.00548195,  0.01185313],
       [-0.11892294,  0.05701514,  0.22170563, ..., -0.01671498,
        -0.00495487,  0.0046973 ]])

Salvando, então, o corpus já processado pelo tf_idf e também a matrix V de LSI em um arquivo texto do tipo Matrix Market format, que permite que a matriz seja guardado em um arquivo texto, mas também pode ser utilizado para cálculos. (o método Serialize guardas os indexes)

In [18]:
gensim.corpora.MmCorpus.serialize('tfidf_model_mm', tfidf_model[corpus])
gensim.corpora.MmCorpus.serialize('lsi_model_mm', lsi_model[tfidf_model[corpus]])

Abrindo as matrizes salvas nos arquivos textos

In [19]:
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')

# * features of td-idf matrix are the different words
print(tfidf_corpus)

# * features of LSI model are the latent topics discovered
print(lsi_corpus)

MmCorpus(43 documents, 1636 features, 3243 non-zero entries)
MmCorpus(43 documents, 43 features, 1849 non-zero entries)


Buscas de Similaridade Semântica utilizando o método Matrix similarity do gensim, o qual computa o "cosine similarity".

Se forem buscados termos que não foram adicionados ao dicionário id2wordDict, não há como mapear esse documento

O parâmetro num_best define quantos dos documentos mais proximos será buscado (Além de normatizar os valores)

Utilizou-se uma query que elimina os documentos extras e seleciona apenas as matérias com similaridade de 10%

In [61]:
from gensim.similarities.docsim import MatrixSimilarity

cosineSimilarity = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)

def search_similarity_query(search_document):

    # * preprocessing and processing until becomes a matrix of type term_to_topic (V)
    doc = preprocess(search_document)
    query_bow = id2wordDict.doc2bow(doc)
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    # * cossine similarity between the vector of the new document vs all other vectors of documents
    # * returns a index-ordered vector with all cosine similarities
    all_ranking = cosineSimilarity[query_lsi]
    
    # * parameter that limits the queries to only search in documents of subjects
    num_subjects = len(subjects_df)
    only_subjects = all_ranking[:num_subjects].tolist()

    # * converting to a list of tuples (id of compared document, similarity)
    ranking = [(i, val) for i, val in enumerate(only_subjects)]

    # * sorting by similarity
    ranking.sort(key= lambda unit: unit[1], reverse=True)

    result = []

    for subject in ranking:
        # * break point, so only subjects with more or equal to 10% similarity are selected
        if round((subject[1] * 100),6) < 10:
            break

        result.append (
            {
                'Relevancia': round((subject[1] * 100),6),
                'Código da Matéria': subjects_df['codigo'][subject[0]],
                'Nome da matéria': subjects_df['nome'][subject[0]]
            }

        )
    
    output = pd.DataFrame(result, columns=['Relevancia','Código da Matéria','Nome da matéria'])
    if len(output) == 0:
        return "O texto procurado não há similaridade com nenhuma das demais"
    return output



Utilizando documentos no formato de str, pode-se realizar buscas de similaridade semântica entre as matérias

In [62]:
search_similarity_query("serie")

,Relevancia,Código da Matéria,Nome da matéria
0,76.909637,MAT0059,METODOS MATEMÁTICOS DA FÍSICA 1
1,63.401717,MAT0026,CÁLCULO 2
2,43.493068,MAT0028,VARIAVEL COMPLEXA 1
3,27.055025,ENE0038,LABORATÓRIO DE CIRCUITOS ELÉTRICOS 2
4,24.504884,ENE0037,CIRCUITOS ELÉTRICOS 2
5,13.096146,IFD0181,FISICA 3 EXPERIMENTAL


Multi-queries:

In [63]:
subsection1_df.head(15)

for i in range(15):
    text = subsection1_df.iloc[i, 8]
    print(f'texto buscado: {text}')
    print(f'texto buscado preprocessado: {preprocess(text)}')
    print( search_similarity_query(text), end='\n\n')


texto buscado: Cálculo diferencial e integral 
texto buscado preprocessado: ['calcular', 'diferencial', 'integral']
   Relevancia Código da Matéria      Nome da matéria
0   43.935215           MAT0025            CÁLCULO 1
1   14.056209           MAT0027            CÁLCULO 3
2   12.869903           MAT0028  VARIAVEL COMPLEXA 1

texto buscado: Cálculo numérico 
texto buscado preprocessado: ['calcular', 'numerico']
   Relevancia Código da Matéria                  Nome da matéria
0   70.257485           ENM0027  TECNOLOGIAS DE COMANDO NUMERICO
1   31.953162           MAT0053                 CALCULO NUMERICO
2   16.215581           MAT0027                        CÁLCULO 3
3   14.343321           MAT0025                        CÁLCULO 1

texto buscado: Álgebra linear 
texto buscado preprocessado: ['algebra', 'linear']
    Relevancia Código da Matéria                   Nome da matéria
0    67.801088           MAT0031       INTRODUCAO A ALGEBRA LINEAR
1    38.838181           CIC0229          